In [1]:
import setup  # notebooks/setup.py
setup.init()

In [2]:
from my_permit import permit_client
permit = permit_client

In [6]:
await permit.api.users.list()

PaginatedResultUserRead(data=[], total_count=0, page_count=0)

In [3]:
resources= [
    {
        "name": "Document",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_discovery",
        "actions": {
            "search": {},
            "detail": {},           
        }
    }
]

In [4]:
# readonly_permissions = [
#     "document:read",
#     "movie_discovery:search"
#     "movie_discovery:detail"
# ]

In [5]:
roles = [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": [
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_discovery:search"
            "movie_discovery:detail"
        ]
    },
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": [
            "document:read",
            "movie_discovery:search"
            "movie_discovery:detail"
        ]
    }
]